In [1]:
import os
import pathlib

import numpy as np
import pandas as pd
import geopandas as gpd

from gtfstools import gtfs
from gtfstools.plotter import plot_gtfs_data

%matplotlib inline
%config InlineBackend.figure_format='retina'

# USER INPUTS

In [2]:
out_folder = os.environ.get('OUT_FOLDER')
out_folder = pathlib.Path(out_folder)
out_folder = out_folder / 'B'

db_folder = os.environ.get('DB_FOLDER')
db_folder = pathlib.Path(db_folder)

In [3]:
path_to_gtfs = (db_folder
                / 'beaga'
                / 'GTFS'
                / '2017'
                / 'GTFS_BH_2017.02.10.zip')

In [4]:
state = 31
cities = 3106200

cutoffs = [0,5,9,12,14,17,19,24]

# PROCESSING

In [5]:
gtfs_feed, routes, trips, stops, stop_times, shapes = gtfs.parse_feed(path_to_gtfs)

C:\Users\brand\anaconda3\envs\pyGeo\lib\site-packages\pyproj\crs\crs.py:53: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [6]:
route_summary = gtfs.summarize_trips(stop_times,
                                     'route_id',
                                     cutoffs,)
route_summary

,route_id,direction,route_name,window,trips,headway_minutes,max_hourly_trips,min_hourly_headway
0,101 01,Inbound,101 Aglomerado Santa Lucia (Principal),00:00 - 05:00,0,NaN,4,15.0
7,101 01,Outbound,101 Aglomerado Santa Lucia (Principal),00:00 - 05:00,0,NaN,0,NaN
1,101 01,Inbound,101 Aglomerado Santa Lucia (Principal),05:00 - 09:00,4,60.0,4,15.0
8,101 01,Outbound,101 Aglomerado Santa Lucia (Principal),05:00 - 09:00,0,NaN,0,NaN
2,101 01,Inbound,101 Aglomerado Santa Lucia (Principal),09:00 - 12:00,0,NaN,4,15.0
...,...,...,...,...,...,...,...,...
7417,SE02 01,Outbound,SE02 Buritis/Savassi (Principal),14:00 - 17:00,0,NaN,0,NaN
7411,SE02 01,Inbound,SE02 Buritis/Savassi (Principal),17:00 - 19:00,0,NaN,4,15.0
7418,SE02 01,Outbound,SE02 Buritis/Savassi (Principal),17:00 - 19:00,0,NaN,0,NaN
7412,SE02 01,Inbound,SE02 Buritis/Savassi (Principal),19:00 - 24:00,0,NaN,4,15.0


In [7]:
cut_routes_df, anomalies = gtfs.cut_routes(stop_times=stop_times,
                                           route_shapes=shapes,
                                           flag_outliers=True,
                                           threshold=2.5,)

Route shapefiles cannot be properly built because gtfs data is incomplete. I'll have to make do with data from other sources

In [ ]:
map_ = plot_gtfs_data(gtfs_data=[route_summary, stop_times, shapes],
                      variable='trips',
                      window='05:00 - 09:00',
                      direction=None,
                      method='NaturalBreaks',
                      k=5,
                      cmap='magma',
                      linear=False,
                      tiles='cartodbpositron',)

map_

In [17]:
import os
import pathlib
import warnings

import geopandas as gpd
import numpy as np
import pandas as pd
import partridge as ptg


%matplotlib inline
%config InlineBackend.figure_format='retina'

In [2]:
out_folder = os.environ.get('OUT_FOLDER')
out_folder = pathlib.Path(out_folder)
out_folder = out_folder / 'B'

db_folder = os.environ.get('DB_FOLDER')
db_folder = pathlib.Path(db_folder)

In [3]:
path_to_gtfs = (db_folder
                / 'beaga'
                / 'GTFS'
                / '2017'
                / 'GTFS_BH_2017.02.10.zip')

In [4]:
with warnings.catch_warnings():
    warnings.filterwarnings('ignore',
                            category=FutureWarning)
    feed = ptg.load_geo_feed(path_to_gtfs)

In [5]:
feed.trips.columns

Index(['route_id', 'service_id', 'trip_id', 'trip_headsign', 'direction_id',
       'block_id', 'shape_id'],
      dtype='object')

In [22]:
feed.routes.columns

Index(['route_id', 'route_short_name', 'route_long_name', 'route_type'], dtype='object')

In [19]:
feed.stop_times.columns

Index(['trip_id', 'arrival_time', 'departure_time', 'stop_id', 'stop_sequence',
       'stop_headsign', 'pickup_type', 'drop_off_type', 'shape_dist_traveled'],
      dtype='object')

In [23]:
feed.stops.columns

Index(['stop_id', 'stop_name', 'stop_desc', 'location_type', 'parent_station',
       'geometry'],
      dtype='object')

In [21]:
routes = feed.routes
trips = feed.trips
stops = feed.stops
stop_times = feed.stop_times
shapes = feed.shapes

C:\Users\brand\anaconda3\envs\pyGeo\lib\site-packages\pyproj\crs\crs.py:68: FutureWarning: '+init=<authority>:<code>' syntax is deprecated. '<authority>:<code>' is the preferred initialization method. When making the change, be mindful of axis order changes: https://pyproj4.github.io/pyproj/stable/gotchas.html#axis-order-changes-in-proj-6
  return _prepare_from_string(" ".join(pjargs))


In [14]:
cu = (feed.trips
.merge(feed.routes, how='left'))

In [31]:
trips = feed.trips.merge(routes, how='left')
stop_times = stop_times.merge(trips, how='left')
stop_times = stop_times.merge(stops, how='left')


In [33]:
stop_times.stop_name

0                                   Estação METRO Vilarinho
1                                    Estação METRO Floramar
2                              Estação METRO Waldomiro Lobo
3                            Estação METRO Primeiro De Maio
4                                 Estação METRO Sao Gabriel
                                ...                        
258427    Ave Jose Candido Da Silveira em frente ao nume...
258428    Ave Jose Candido Da Silveira em frente ao nume...
258429    Ave Jose Candido Da Silveira em frente ao nume...
258430    Ave Jose Candido Da Silveira em frente ao nume...
258431     Rua Gustavo Da Silveira em frente ao numero 1825
Name: stop_name, Length: 258432, dtype: object

In [26]:
trips_and_routes = pd.merge(feed.trips,
                            feed.routes,
                            how='left',)
operational_data = 


test = (
      feed.trips
      .merge(feed.routes, how='left')
      .merge(feed.trips, how='left')
      .merge(feed.stops, how='left')
     )

MergeError: No common columns to perform merge on. Merge options: left_on=None, right_on=None, left_index=False, right_index=False

In [28]:
warnings.filterwarnings?

Signature:
warnings.filterwarnings(
    action,
    message='',
    category=<class 'Warning'>,
    module='',
    lineno=0,
    append=False,
)
Docstring:
Insert an entry into the list of warnings filters (at the front).

'action' -- one of "error", "ignore", "always", "default", "module",
            or "once"
'message' -- a regex that the warning message must match
'category' -- a class that the warning must be a subclass of
'module' -- a regex that the module name must match
'lineno' -- an integer line number, 0 matches all warnings
'append' -- if true, append to the list of filters
File:      c:\users\brand\anaconda3\envs\pygeo\lib\warnings.py
Type:      function
